In [1]:
%load_ext autoreload
%autoreload 2
from pydarknet.libdarknet import Libdarknet

In [2]:
import ctypes
import os

In [3]:
self = Libdarknet()

In [4]:
self

In [5]:
self.lib

<CDLL '/mnt/ubuntu1604_2/home/jed/.darknet/libdarknet.so', handle 2d8e760 at 0x7facb83899e8>

In [6]:
self.exists

True

In [7]:
lib = self.lib

In [33]:
# Relative files are **relative to the .darknet directory!
metadata = self.get_metadata("cfg/coco.data")

network = self.load_network("cfg/yolov3.cfg", "weights/yolov3.weights", 0)
img = self.load_image_color("data/dog.jpg")

In [9]:
from pydarknet.libdarknet.structs import Image, Detection

In [10]:
self.lib.network_predict_image.argtypes = [ctypes.c_void_p, Image]
self.lib.network_predict_image.restype = ctypes.POINTER(ctypes.c_float)

self.lib.network_predict_image(network, img)

In [11]:
net_output = self.network_predict_image(network, img)

In [12]:
net_output[0]

0.24767105281352997

In [16]:
self.get_network_boxes(network, img)

(c_int(13), <pydarknet.libdarknet.LP_Detection at 0x7facb83b1730>)

In [17]:
num = ctypes.c_int(0)
pnum = ctypes.pointer(num)

In [18]:
dets = self.lib.get_network_boxes(network, img.w, img.h, 0.5, 0.5, None, 0, pnum)

In [29]:
for idx in range(num.value):
    det=dets[idx]
    print(det.objectness, det.prob[0])

0.7897613644599915 0.0
0.998834490776062 0.0
0.8852115869522095 0.0
0.6225367784500122 0.0
0.5576186180114746 0.0
0.9941537976264954 0.0
0.9076856970787048 0.0
0.6667616963386536 0.0
0.5439086556434631 0.0
0.6891421675682068 0.0
0.7975820302963257 0.0
0.9996165633201599 0.0
0.9953235387802124 0.0


In [30]:
det.classes

80

In [24]:
thresh=0.5
heir_thresh=0.5

In [44]:
self.lib.get_network_boxes.argtypes = [
    ctypes.c_void_p,
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_float,
    ctypes.c_float,
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int, 
    ctypes.POINTER(ctypes.c_int),
]
self.lib.get_network_boxes.restype = ctypes.POINTER(Detection)

num = ctypes.c_int(0)
pnum = ctypes.pointer(num)        
dets = self.lib.get_network_boxes(network, img.w, img.h, thresh, heir_thresh, None, 0, pnum)

In [32]:
dets

In [45]:
pnum[0]

13

In [46]:
num.value

13

In [47]:
num = pnum[0]
num

13

In [48]:
det=dets[4]

In [49]:
det

Detection<Box<317.7, 290.0, 450.0, 329.5>, 0.56>

In [50]:
det.classes

80

In [ ]:
from pydarknet.libdarknet.structs import Detection

In [ ]:
nms=0.5

In [ ]:
self.lib.do_nms_sort.argtypes = [ctypes.POINTER(Detection), ctypes.c_int, ctypes.c_int, ctypes.c_float]
self.lib.do_nms_sort(dets, num, metadata.classes, nms);

In [ ]:
dets[0]

In [ ]:
dets[:num-1]

In [ ]:
d = dets[0]

In [ ]:
dets[num-1].bbox

In [ ]:
d.classes

In [ ]:
d.objectness

In [ ]:
d.sort_class

In [ ]:
dets[0:num-1]

In [ ]:
dets

In [ ]:
pnum

In [52]:
dets = self.get_network_boxes(network, img)

In [31]:
dets

In [55]:
self.free_image(img)